# Tweets from bioRxiv and medRxiv

Find and display events from the CSHL DOI prefix 10.1101 from the last 7 days, and find the most tweeted preprints.

In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '..')

import pandas as pd # data analysis library
import json
import mrced2 # module to run event data queries
import os # some file manipulations
import math # some number manipulations
import altair.vegalite.v3 as alt # some data visualizations
from IPython.display import Markdown as md # some markdown manipulations
from datetime import datetime, date, timedelta # some date manipulations

In [2]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
end_date = date.today()

In [3]:
ed = mrced2.eventData(email = email)
ed.buildQuery({'obj-id.prefix' : prefix, 'source': 'twitter', 'rows': 0,'from-occurred-date' : start_date, 'until-occurred-date' : end_date})

https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&obj-id.prefix=10.1101&source=twitter&rows=0&from-occurred-date=2021-05-24&until-occurred-date=2021-05-31


In [4]:
ed.runQuery(retry = 5)

Event Data query started...
API query complete  200
output file written to 1101/tweets.json


In [5]:
pages = math.ceil(ed.events.getHits() / 1000)

13160 events found


In [6]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

# find the all result pages for the search
ed = mrced2.eventData(email = email)
ed.getAllPages(pages, {'rows': 1000, 'obj-id.prefix' : prefix, 'source': 'twitter', 'from-occurred-date' : start_date, 'until-occurred-date' : end_date}, fileprefix = '1101/tweets_') 

https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-05-24&until-occurred-date=2021-05-31
Event Data query started...
API query complete  200
output file written to 1101/tweets_0000.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=55620f58-583c-4350-a3e3-716ed0a187c4&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-05-24&until-occurred-date=2021-05-31
Event Data query started...
API query complete  200
output file written to 1101/tweets_0001.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=6209428e-2a42-45c9-bb01-dd044228679c&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-05-24&until-occurred-date=2021-05-31
Event Data query started...
API query complete  200
output file written to 1101/tweets_0002.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=9c227615-7f39-4963-ae56-6185f6cda173&rows=

Initialisation to look at the properties of the results.

In [7]:
jd1 = mrced2.eventRecord() # instance of a class to interpret the events
files = os.listdir('1101') # get all the filenames

jd1.mergeJsons(files, folder = '1101') # load the json event data from multiple files

failed to load preprint_tweets_2021-05-10.csv
failed to load preprint_tweets_2021-05-17.csv
failed to load .DS_Store
failed to load .gitkeep
failed to load tweets_sorted.json
failed to load preprint_tweets_2021-05-18.csv
failed to load preprint_tweets_2021-05-24.csv
failed to load preprint_tweets_2021-05-09.csv
failed to load preprint_tweets_2021-05-08.csv
output file written to 1101/tweets.json


In [8]:
js = json.load(open("1101/tweets.json"))
df = pd.json_normalize(js, record_path = ['message', 'events'])
gdf = df.groupby(['obj_id']).size().reset_index(name='count').sort_values('count', ascending=False)
cdf = gdf[gdf['count'] >= 5]

In [9]:
email = "info@front-matter.io"
rest = mrced2.restApi(email = email)

data = []

for index, row in cdf.iterrows():
    rest.runQuery(row)
    if rest.work is not None and date.fromisoformat(rest.work["posted"]) >= start_date:
        data.append(rest.work)
    
tdf = pd.DataFrame(data, columns=['doi','tweets','archive','subject-area','covid','title','authors','abstract','posted'])
tdf.to_csv('1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv')

tdf.head(50)

REST API query started for 10.1101/2021.05.21.21257595...
REST API query complete  200
REST API query started for 10.1101/2021.05.24.445508...
REST API query complete  200
REST API query started for 10.1101/2020.12.12.422516...
REST API query complete  200
REST API query started for 10.1101/2021.03.04.430128...
REST API query complete  200
REST API query started for 10.1101/2020.11.15.383323...
REST API query complete  200
REST API query started for 10.1101/2021.05.22.21257633...
REST API query complete  200
REST API query started for 10.1101/2021.05.28.446163...
REST API query complete  200
REST API query started for 10.1101/2021.01.22.427830...
REST API query complete  200
REST API query started for 10.1101/2021.05.22.21257658...
REST API query complete  200
REST API query started for 10.1101/2021.01.21.21250266...
REST API query complete  200
REST API query started for 10.1101/2021.03.05.21252960...
REST API query complete  200
REST API query started for 10.1101/2021.01.27.21250617.

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
0,10.1101/2021.05.21.21257595,1122,medRxiv,Infectious Diseases (except HIV/AIDS),True,Ivermectin for the treatment of COVID-19: A sy...,"[{'name': 'Yuani M. Roman'}, {'name': 'Paula A...",<sec><title>Background</title><p>We systematic...,2021-05-25
1,10.1101/2021.05.24.445508,1064,bioRxiv,Pharmacology and Toxicology,False,Effects of vapor exposure to Δ9-tetrahydrocann...,"[{'name': 'Arnold Gutierrez'}, {'name': 'Kevin...",<sec><title>Rationale</title><p>Despite a long...,2021-05-25
2,10.1101/2021.05.22.21257633,392,medRxiv,Epidemiology,True,Genomic reconstruction of the SARS-CoV-2 epide...,"[{'name': 'Harald S. Vöhringer'}, {'name': 'Th...",<p>Despite regional successes in controlling t...,2021-05-26
3,10.1101/2021.05.28.446163,371,bioRxiv,Microbiology,True,The SARS-CoV-2 variants associated with infect...,"[{'name': 'Thomas P. Peacock'}, {'name': 'Caro...",<p>The SARS-CoV-2 spike (S) glycoprotein conta...,2021-05-28
4,10.1101/2021.05.22.21257658,308,medRxiv,Epidemiology,True,Effectiveness of COVID-19 vaccines against the...,"[{'name': 'Jamie Lopez Bernal'}, {'name': 'Nic...",<sec><title>Background</title><p>The B.1.617.2...,2021-05-24
5,10.1101/2021.05.24.444987,188,bioRxiv,Animal Behavior and Cognition,False,Distinct movement patterns generate stages of ...,"[{'name': 'Abel Corver'}, {'name': 'Nicholas W...",<p>The geometric complexity and stereotypy of ...,2021-05-25
6,10.1101/2021.05.25.21257823,131,medRxiv,Epidemiology,True,Estimating the wave 1 and wave 2 infection fat...,"[{'name': 'Soumik Purkayastha'}, {'name': 'Rit...",<sec><title>Objective</title><p>There has been...,2021-05-27
7,10.1101/2021.05.21.445189,90,bioRxiv,Immunology,True,Evaluation of mRNA-1273 against SARS-CoV-2 B.1...,"[{'name': 'Kizzmekia S. Corbett'}, {'name': 'A...",<sec><title>Background</title><p>Vaccine effic...,2021-05-24
8,10.1101/2021.05.24.445471,90,bioRxiv,Genetics,False,Multiple Causal Variants Underlie Genetic Asso...,"[{'name': 'Nathan S. Abell'}, {'name': 'Marian...",<p>The majority of associations between geneti...,2021-05-25
9,10.1101/2021.05.24.445457,62,bioRxiv,Genetics,False,Island-specific evolution of a sex-primed auto...,"[{'name': 'Longhua Guo'}, {'name': 'Joshua Blo...",<p>The sexual biotype of the planarian <italic...,2021-05-24


### Tweets of bioRxiv and medRxiv preprints

In [10]:
num_rows = tdf['archive'].count()
num_covid = tdf['covid'].value_counts(ascending=True)[1]
num_biorxiv = tdf['archive'].value_counts(ascending=True)[1]
num_medrxiv = tdf['archive'].value_counts(ascending=True)[0]
end_date = date.today().strftime('%Y-%m-%d')
max_count = tdf['tweets'].max()

md('{} preprints (including {} covering SARS-CoV-2, {} from bioRxiv and {} from medRxiv) published in the last 7 days before {} had been tweeted at least 5 times (maximum {}).'.format(num_rows, num_covid, num_biorxiv, num_medrxiv, end_date, max_count))

92 preprints (including 21 covering SARS-CoV-2, 79 from bioRxiv and 13 from medRxiv) published in the last 7 days before 2021-05-31 had been tweeted at least 5 times (maximum 1122).